# Demo: Draft model + sklearn

<a href="https://colab.research.google.com/github/basetenlabs/demos/blob/main/Draft_Model_Demo_Sklearn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook presents a workflow with live reload: deploying your changes in seconds to a production-like environment for rapid iteration on your model serving code.

All you'll need is a Baseten account and your API key!

In [ ]:
%pip install --upgrade baseten sklearn truss palmerpenguins

In [ ]:
# First, we train our model. Thanks to https://github.com/mcnakhaee/palmerpenguins for the training code!

from sklearn.model_selection import train_test_split
from sklearn.pipeline import FeatureUnion, make_pipeline
from sklearn.metrics import confusion_matrix
### To deal with missing values
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.tree import DecisionTreeClassifier
### Penguins!!
from palmerpenguins import load_penguins

data, target = load_penguins(return_X_y = True)
imp = IterativeImputer(max_iter=10, random_state=0)
clf = make_pipeline(imp, DecisionTreeClassifier())
clf = clf.fit(data, target)

In [ ]:
# Then, let's package the model as a Truss
import truss

truss.mk_truss(clf, "penguin-classifier")

In [ ]:
# Then, we deploy it as a draft model

import baseten

api_key = "YOUR API KEY HERE"
baseten.login(api_key)

packaged_model = truss.from_directory("penguin-classifier")
baseten.deploy(
    packaged_model,
    model_name="Penguin Predictor",
    is_draft=True
)

In [ ]:
# Once the model is deployed (which will take a few minutes), we can invoke it

deployed_model_id = "VERSION ID" # See draft model to find version ID
model_input = {"inputs": [[40, 20, 200, 4000]]}

deployed_model = baseten.deployed_model_version_id(deployed_model_id)
deployed_model.predict(model_input)

## Updating model input

Now that we have deployed and invoked the model, we might want to adjust the model serving code. For example, let's adjust the model input format.

Right now, it takes an input dictionary with a list of lists. Let's say you're developing this model to work with an existing applications, whose requirements state the model input must be formatted as follows:

```json
{
  "bill_length": 40,
  "bill_depth": 20,
  "flipper_length": 200,
  "body_mass": 4000
}
```

We'll open up `penguin-classifier/model/model.py` and implement the `preprocess()` function with the following:

```python
def preprocess(self, request: Dict) -> Dict:
    request = {"inputs": [[
      request["bill_length"],
      request["bill_depth"],
      request["flipper_length"],
      request["body_mass"]
    ]]}
    return request
```

After saving our changes, we'll reload the Truss and update the draft model. We'll call the newly deployed model right away as the model code will be reloaded live.

In [ ]:
# This time, deploying the updated model should take just seconds

packaged_model = truss.from_directory("penguin-classifier")
baseten.deploy(
    packaged_model,
    model_name="Penguin Predictor",
    is_draft=True
)

In [ ]:
# Invoke the model again with new input format that matches the requirements

model_input = {
  "bill_length": 40,
  "bill_depth": 20,
  "flipper_length": 200,
  "body_mass": 4000
}

deployed_model = baseten.deployed_model_version_id(deployed_model_id)
deployed_model.predict(model_input)

## Formatting model output

Turns out, the frontend team wants to return the model output to the user directly, and asks you to format the prediction as a string. No problem! You can test it out without waiting for your model to rebuild with live reload.

We'll open up `penguin-classifier/model/model.py` again and fill in the `postprocess()` function with the following:

```python
def postprocess(self, request: Dict) -> Dict:
    return f"It appears as though you've discovered a {request['predictions'][0]} penguin!"

```

After saving our changes, we'll once again reload the Truss and update the draft model.

In [ ]:
# Again, deploying the updated model should take just seconds

packaged_model = truss.from_directory("penguin-classifier")
baseten.deploy(
    packaged_model,
    model_name="Penguin Predictor",
    is_draft=True
)

In [ ]:
# Invoke the model again with to get a user-friendly string

model_input = {
  "bill_length": 40,
  "bill_depth": 20,
  "flipper_length": 200,
  "body_mass": 4000
}

deployed_model = baseten.deployed_model_version_id(deployed_model_id)
deployed_model.predict(model_input)

## Deploying the final model

Now that we are happy with our model, we can deploy it and it will no longer be a draft. Simply remove `is_draft=True` from the deployment script. Deploying the model for real doesn't use live reload, so take a break after all of your hard work and come back in a few minutes to invoke the model version with a new version ID.

In [ ]:
baseten.deploy(
    packaged_model,
    model_name="Penguin Predictor"
)

In [ ]:
# Invoke the fully deployed model

deployed_model_id = "NEW VERSION ID" # See model version to find version ID
model_input = {
  "bill_length": 40,
  "bill_depth": 20,
  "flipper_length": 200,
  "body_mass": 4000
}

deployed_model = baseten.deployed_model_version_id(deployed_model_id)
deployed_model.predict(model_input)